In [1]:
import pandas as pd
import warnings

from datasets import load_dataset
from transformers import (
    AutoTokenizer, DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model
import torch

In [2]:
torch.manual_seed(0)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
def print_trainable_parameters(model):
    tot_params = 0
    trainable_params = 0
    for name, param in model.named_parameters():
        tot_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {tot_params:,} || trainable: {100 * trainable_params / tot_params:.2f}%"
    )


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return {"accuracy": (predictions == labels).mean()}


def show_example(idx, tokenized_dataset, id2label):
    txt = tokenized_dataset[idx]["text"]
    ids = tokenized_dataset[idx]['input_ids']
    label = tokenized_dataset[idx]['label']
    print(f"Text: {txt}")
    print(f"Tokenized: {ids}")
    print(f"Label: {label} -> {id2label[label]}")

# Dataset

In [4]:
# Load Dataset
dataset = load_dataset("dair-ai/emotion")

id2label = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
label2id = {v: k for k, v in id2label.items()}

train_ds = dataset["train"]
val_ds = dataset["validation"]
test_ds = dataset["test"]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2", 
    num_labels=len(id2label),
    label2id=label2id,
    id2label=id2label
)

print(model)
print_trainable_parameters(model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)
trainable params: 124,444,416 || all params: 124,444,416 || trainable: 100.00%


In [7]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

print(f"Training Size: {len(tokenized_train_ds)}")
print(f"Validation Size: {len(tokenized_val_ds)}")
print(f"Test Size: {len(tokenized_test_ds)}")

Training Size: 16000
Validation Size: 2000
Test Size: 2000


In [8]:
show_example(0, tokenized_train_ds, id2label)

Text: i didnt feel humiliated
Tokenized: [72, 42547, 1254, 42659, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]
Label: 0 -> sadness


# Evaluate Base Model

In [9]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer
)

In [10]:
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define the evaluation function
def evaluate(model, dataset, batch_size=8):
    model.eval()
    all_preds = []
    all_labels = []
    all_pred_labels = []
    
    N = len(dataset)
    pbar = tqdm(total=N//batch_size)

    for batch in DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
        
        logits_array = logits.cpu().numpy()
        all_preds.extend(logits_array)
        all_labels.extend(labels.cpu().numpy())
        all_pred_labels.extend(np.argmax(logits_array, axis=1))
        pbar.update(1)
        pbar.set_description(f"batch {pbar.n}/{pbar.total}")

    pbar.close()
    
    acc = compute_metrics((np.array(all_preds), np.array(all_labels)))
    return acc, all_pred_labels, all_labels

In [11]:
model.to(device)
print(f"device: {model.device}")

# Evaluate the original GPT-2 model
tokenized_test_dataset = tokenized_test_ds.remove_columns(["text"])
tokenized_test_dataset.set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])

original_acc, _, _ = evaluate(model, tokenized_test_dataset)
print(f"Original GPT-2 model accuracy: {original_acc}")

device: cuda:0


batch 250/250: 100%|██████████| 250/250 [00:05<00:00, 42.05it/s]

Original GPT-2 model accuracy: {'accuracy': 0.036}


# Load Models

In [12]:
from peft import TaskType
import time

def train_peft_lora(X_train, X_val, r=1, epochs=5, batch_size=16, disable_tqdm=False, save=True):
    warnings.filterwarnings("ignore")
    t_start = time.time()
    # create a peft model
    lora_config = LoraConfig(
        r=r,
        lora_alpha=0.8,
        bias="none",
        lora_dropout=0.1,
        fan_in_fan_out=True,
        task_type=TaskType.SEQ_CLS
    )
    # create a peft model
    lora_model = get_peft_model(
        model=model,
        peft_config=lora_config
    ).to(device)
    # extract parameters
    tot_params = sum(p.numel() for p in lora_model.parameters())
    trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
    perc_trainable = 100 * trainable_params / tot_params
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./model_dir/lora_r{r}",
        logging_dir=f"./model_dir/logs/lora_r{r}",
        learning_rate=5e-4,
        weight_decay=0.01,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        save_total_limit=1,
        save_strategy="epoch",
        eval_strategy="epoch",
        logging_steps=1000,
        use_cpu=False,
        disable_tqdm=disable_tqdm,
        seed=110
    )
    # LoRA uses "labels", not "label". We need to rename the datasets.
    train_lora = X_train.rename_column('label', 'labels')
    val_lora = X_val.rename_column('label', 'labels')
    # Trainer object
    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_lora,
        eval_dataset=val_lora,
        compute_metrics=compute_metrics,
        data_collator=data_collator
    )
    # Train the model
    trainer.train()

    if save:
        trainer.save_model(f"./model_dir/fine_tuned_model/lora_r{r}")
        #tokenizer.save_pretrained(f"./model_dir/fine_tuned_model/lora_r{r}")
        #torch.save(training_args, f"./model_dir/fine_tuned_model/lora_r{r}/training_args.bin")
        #trainer.state.save_to_json(f"./model_dir/fine_tuned_model/lora_r{r}/trainer_state.json")

    # evaluate the model
    acc = trainer.evaluate()['eval_accuracy']
    t_end = time.time()
    output = {
        'r': r,
        'trainable_params': trainable_params,
        'tot_params': tot_params,
        'perc_trainable': perc_trainable,
        'accuracy': acc,
        'time': t_end - t_start
    }
    return output

In [18]:
output_lora_perfs = {}

# Train the model with LoRA
r_vals = [1, 2, 4, 8, 16]

for r in r_vals:
    output_lora_perfs[r] = train_peft_lora(
    X_train=tokenized_train_ds, 
    X_val=tokenized_val_ds, 
    r=r, 
    epochs=5, 
    batch_size=32,
    save=True,
    disable_tqdm=False
)
    
output_lora_perfs = pd.DataFrame(output_lora_perfs).T
output_lora_perfs.to_csv("lora_perfs.csv")

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.40435791015625, 'eval_accuracy': 0.8545, 'eval_runtime': 5.7253, 'eval_samples_per_second': 349.325, 'eval_steps_per_second': 11.004, 'epoch': 1.0}
{'loss': 0.6974, 'grad_norm': 3.1234071254730225, 'learning_rate': 0.0003, 'epoch': 2.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2844401001930237, 'eval_accuracy': 0.903, 'eval_runtime': 5.7064, 'eval_samples_per_second': 350.483, 'eval_steps_per_second': 11.04, 'epoch': 2.0}


...

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2251196801662445, 'eval_accuracy': 0.9155, 'eval_runtime': 35.8127, 'eval_samples_per_second': 55.846, 'eval_steps_per_second': 1.759, 'epoch': 4.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2196282297372818, 'eval_accuracy': 0.914, 'eval_runtime': 36.0373, 'eval_samples_per_second': 55.498, 'eval_steps_per_second': 1.748, 'epoch': 5.0}
{'train_runtime': 1591.5876, 'train_samples_per_second': 50.264, 'train_steps_per_second': 1.571, 'train_loss': 0.4568334655761719, 'epoch': 5.0}


  0%|          | 0/63 [00:00<?, ?it/s]

In [91]:
perf = train_peft_lora(
    X_train=tokenized_train_ds, 
    X_val=tokenized_val_ds, 
    r=1, 
    epochs=5, 
    batch_size=32,
    disable_tqdm=False,
    save=True
)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.404478520154953, 'eval_accuracy': 0.8565, 'eval_runtime': 4.9422, 'eval_samples_per_second': 404.679, 'eval_steps_per_second': 12.747, 'epoch': 1.0}
{'loss': 0.6972, 'grad_norm': 3.652996778488159, 'learning_rate': 0.0003, 'epoch': 2.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2811495363712311, 'eval_accuracy': 0.9045, 'eval_runtime': 4.9428, 'eval_samples_per_second': 404.631, 'eval_steps_per_second': 12.746, 'epoch': 2.0}


...

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2269621193408966, 'eval_accuracy': 0.9175, 'eval_runtime': 4.84, 'eval_samples_per_second': 413.224, 'eval_steps_per_second': 13.017, 'epoch': 4.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.22183148562908173, 'eval_accuracy': 0.919, 'eval_runtime': 4.8239, 'eval_samples_per_second': 414.605, 'eval_steps_per_second': 13.06, 'epoch': 5.0}
{'train_runtime': 471.0543, 'train_samples_per_second': 169.832, 'train_steps_per_second': 5.307, 'train_loss': 0.458707666015625, 'epoch': 5.0}


  0%|          | 0/63 [00:00<?, ?it/s]

# Inference

In [92]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained("./model_dir/fine_tuned_model/lora_r1")
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels=len(id2label))
# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))
# set the pad token
model.config.pad_token_id = tokenizer.pad_token_id
# load the peft model
lora_model = PeftModel.from_pretrained(model, "./model_dir/fine_tuned_model/lora_r1").to(device)
# Inference using the trained model
acc, all_pred_labels, all_labels = evaluate(lora_model, tokenized_test_dataset, batch_size=32)

print(f"LoRA model accuracy: {acc}")

# Prepare the prediction table
examples_df = pd.DataFrame(tokenized_test_ds.select_columns(["text", "label"]))
examples_df['emotion'] = examples_df['label'].map(id2label)
examples_df['predicted_label'] = all_pred_labels
examples_df['predicted_emotion'] = examples_df['predicted_label'].map(id2label)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
batch 63/62: : 63it [00:04, 13.00it/s]

LoRA model accuracy: {'accuracy': 0.904}


In [104]:
# Stratified sampling
pd.set_option('display.max_colwidth', None)

examples_df.groupby('emotion').apply(lambda x: x.sample(2, random_state=110)).reset_index(drop=True)

,text,label,emotion,predicted_label,predicted_emotion
0,i feel irritable about the number of people that came into our office whining about their own circumstances i realize im not practicing thinking about the good things and i find it a better way to pull yourself into the present,3,anger,3,anger
1,i feel appalled right now,3,anger,3,anger
2,i feel that i am afraid of whatever ad anything that will happen and idc is it good or bad i am just afraid and i hope god you will help me in whatever i do,4,fear,4,fear
3,i guess feelings aren t meant to be inhibited or prohibited,4,fear,0,sadness
4,i bought this doraemon backpack from a charity store i had every intention of putting it in my etsy store but i feel like its too cute to sell,1,joy,1,joy
5,i feel for this divine landmass and all the respect i bear in my heart for the greatness residing on it,1,joy,1,joy
6,i feel that i am supporting the troops by demanding that we not send our young men and women into harm s way to bear arms against a country that has done nothing to threaten us at any point,2,love,2,love
7,i feel very tender for anyone who is upset by the bee movie sort of like how you feel about old aunts who dont realize how prickly their whiskers are getting slightly repulsed but very sad for their decline,2,love,2,love
8,i came home waiting for the shower read something which made me upset thats why i feel discontent haha,0,sadness,3,anger
9,im starting to feel really pathetic giving the bulk of my enthusiasm these days to the kardashians us weekly and roseanne marathons and completely ignoring this blog,0,sadness,0,sadness
